# Analyse X_train_preprocessed

Notebook d'analyse des données preprocessées pour valider les transformations appliquées.

In [1]:
import polars as pl
import numpy as np
from pathlib import Path
import glob

## 1. Chargement des données

In [2]:
# Trouver le dernier fichier preprocessed
data_path = Path('../../data')
preprocessed_files = sorted(data_path.glob('X_train_preprocessed_*.csv'))

if not preprocessed_files:
    raise FileNotFoundError("Aucun fichier X_train_preprocessed_*.csv trouvé")

latest_file = preprocessed_files[-1]
print(f"Fichier chargé: {latest_file.name}")

# Charger les données
df_preprocessed = pl.read_csv(latest_file)
df_original = pl.read_csv(data_path / 'X_train.csv')

Fichier chargé: X_train_preprocessed_20251129_153225.csv


## 2. Statistiques générales

In [13]:
print("="*80)
print("STATISTIQUES GÉNÉRALES")
print("="*80)

print("\n📊 DIMENSIONS")
print(f"  Données originales:     {df_original.shape}")
print(f"  Données preprocessées:  {df_preprocessed.shape}")
print(f"  Réduction colonnes:     {df_original.shape[1] - df_preprocessed.shape[1]} ({(df_original.shape[1] - df_preprocessed.shape[1])/df_original.shape[1]*100:.1f}%)")

print("\n📉 VALEURS MANQUANTES")
null_count_original = df_original.null_count().sum_horizontal()[0]
null_count_preprocessed = df_preprocessed.null_count().sum_horizontal()[0]
total_cells_original = df_original.shape[0] * df_original.shape[1]
total_cells_preprocessed = df_preprocessed.shape[0] * df_preprocessed.shape[1]

print(f"  Original:      {null_count_original:,} ({null_count_original/total_cells_original*100:.2f}%)")
print(f"  Preprocessé:   {null_count_preprocessed:,} ({null_count_preprocessed/total_cells_preprocessed*100:.2f}%)")
print(f"  Amélioration:  {null_count_original - null_count_preprocessed:,} valeurs manquantes en moins")

STATISTIQUES GÉNÉRALES

📊 DIMENSIONS
  Données originales:     (1172086, 307)
  Données preprocessées:  (1172086, 86)
  Réduction colonnes:     221 (72.0%)

📉 VALEURS MANQUANTES
  Original:      213,250,711 (59.26%)
  Preprocessé:   32,971,221 (32.71%)
  Amélioration:  180,279,490 valeurs manquantes en moins


In [4]:
# Afficher aperçu
print("\n📋 APERÇU DES DONNÉES PREPROCESSÉES (5 premières lignes)")
df_preprocessed.head(5)


📋 APERÇU DES DONNÉES PREPROCESSÉES (5 premières lignes)


,Year,CNT,CNTRYID,CNTSCHID,CNTSTUID,STRATUM,OECD,ADMINMODE,LANGTEST_COG,ST003D02T,ST004D01T,EFFORT1,EFFORT2,AGE,GRADE,IMMIG,COBN_S,WB165,IC171,ST038,ST034,PA042,IC173,WB154,IC174,ST021,PA183,WB163,PA166,ST250,ST296,ST270,ST353,ST349,WB164,ST062,…,math_q5_average_score,math_q6_average_score,math_q7_average_score,math_q8_average_score,math_q9_average_score,science_q1_average_score,science_q2_average_score,science_q3_average_score,science_q4_average_score,science_q5_average_score,science_q6_average_score,science_q7_average_score,science_q8_average_score,reading_q1_total_timing,reading_q2_total_timing,reading_q3_total_timing,reading_q4_total_timing,reading_q5_total_timing,reading_q6_total_timing,math_q1_total_timing,math_q2_total_timing,math_q3_total_timing,math_q4_total_timing,math_q5_total_timing,math_q6_total_timing,math_q7_total_timing,math_q8_total_timing,math_q9_total_timing,science_q1_total_timing,science_q2_total_timing,science_q3_total_timing,science_q4_total_timing,science_q5_total_timing,science_q6_total_timing,science_q7_total_timing,OCOD1_grouped,OCOD2_grouped
i64,i64,str,f64,f64,f64,str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,…,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,i64,i64
384002,2022,"""NLD""",528.0,5.2800132e7,5.2801144e7,"""NLD06""",1.0,2.0,322.0,2.0,1.0,8.0,8.0,15.75,0.0,2.0,52800.0,1.0,0.0,0.0,2.333333,0.0,0.0,2.0,0.0,0.0,0.0,6.5,0.0,4.0,1.0,1.666667,0.0,0.0,0.0,0.333333,…,0.0,1.0,0.0,0.5,1.0,0.0,0.5,0.0,0.333333,0.0,0.666667,0.5,0.333333,null,null,null,null,null,null,18736.0,23055.25,162720.0,110641.333333,96368.333333,71813.5,98875.0,63689.4,37232.0,84299.333333,90156.0,66612.0,73547.6,68912.0,56623.25,104190.5,3,2
1118072,2018,"""QAZ""",31.0,3.100106e6,3.100424e6,"""QAZ0101""",0.0,2.0,803.0,11.0,2.0,7.0,6.0,15.42,0.0,null,3100.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6.0,0.0,1.666667,0.0,0.0,0.0,0.0,…,0.0,0.0,0.5,0.0,0.0,null,null,null,null,null,null,null,null,70444.75,10035.75,4873.799805,67025.601562,78748.429688,8368.28418,50234.0,80788.0,48059.0,116718.0,61003.0,34117.0,56901.667969,76615.0,21500.333984,null,null,null,null,null,null,null,2,9
845454,2018,"""FRA""",250.0,2.500001e7,2.5005207e7,"""FRA0101""",1.0,2.0,493.0,5.0,1.0,null,null,16.0,0.0,null,25000.0,0.0,3.5,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,13.0,1.166667,1.333333,0.0,3.25,0.0,0.0,…,null,null,null,null,null,1.0,1.0,0.0,1.0,0.666667,1.0,1.0,1.0,110500.601562,64958.0,35507.601562,63326.144531,4517.952148,39792.332031,null,null,null,null,null,null,null,null,null,81443.664062,190243.40625,36257.0,148073.5,2121.249756,90540.335938,57954.0,3,3
1728613,2015,"""QES""",971.0,9.710024e7,9.7127584e7,"""ESP1633""",2.0,2.0,156.0,3.0,2.0,null,null,16.17,null,1.0,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,…,null,null,null,null,null,1.0,1.0,0.5,1.0,1.0,1.0,0.5,1.0,79739.75,68604.664062,53565.5,61712.332031,63600.5,114817.335938,null,null,null,null,null,null,null,null,null,62224.667969,83289.664062,91510.75,100649.5,82378.5,74348.25,91230.664062,null,null
1083243,2018,"""PHL""",608.0,6.0800071e7,6.0802698e7,"""PHL0011""",0.0,2.0,313.0,9.0,1.0,10.0,10.0,15.5,-1.0,null,60800.0,0.0,0.0,6.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,8.0,0.0,4.0,0.0,0.0,0.0,0.333333,…,0.0,1.0,0.5,0.0,0.333333,null,null,null,null,null,null,null,null,65393.0,57448.0,75731.5,126398.601562,54334.800781,4223.181152,137692.0,36046.0,43095.75,82282.5,121010.5,191303.0,77024.335938,197619.671875,64659.667969,null,null,null,null,null,null,null,9,7


In [5]:
# Info sur les colonnes
print("\n📋 TYPES DE DONNÉES")
dtypes = df_preprocessed.schema
dtype_counts = {}
for dtype in dtypes.values():
    dtype_str = str(dtype)
    dtype_counts[dtype_str] = dtype_counts.get(dtype_str, 0) + 1

for dtype, count in sorted(dtype_counts.items(), key=lambda x: x[1], reverse=True):
    print(f"  {dtype}: {count} colonnes")


📋 TYPES DE DONNÉES
  Float64: 80 colonnes
  Int64: 4 colonnes
  String: 2 colonnes


## 3. Vérification des transformations

### 3.1 Variables supprimées

In [ ]:
# Variables redondantes ordinales qui devaient être supprimées
redundant_ordinal = ['ST005', 'ST007', 'ST253', 'ST255', 'ST097']

# Variables métadonnées catégorielles qui devaient être supprimées
metadata_categorical = [
    'Option_CT', 'Option_FL', 'Option_ICTQ', 'Option_PQ',
    'Option_TQ', 'Option_UH', 'Option_WBQ',
    'CYC', 'NatCen', 'SUBNATIO'
]

# Variables redondantes catégorielles qui devaient être supprimées
redundant_categorical = [
    'LANGTEST_PAQ', 'LANGTEST_QQQ', 'ST003D03T', 'ST001D01T',
    'PA008', 'PA162', 'OCOD3'
]

print("="*80)
print("VÉRIFICATION DES SUPPRESSIONS")
print("="*80)

print("\n🔢 Variables ordinales redondantes (devaient être supprimées):")
for var in redundant_ordinal:
    if var in df_original.columns:
        status = "❌ PRÉSENTE" if var in df_preprocessed.columns else "✅ SUPPRIMÉE"
        print(f"  {var}: {status}")
    else:
        print(f"  {var}: ⚠️  Absente du dataset original")

print("\n🏷️  Variables métadonnées catégorielles (devaient être supprimées):")
for var in metadata_categorical:
    if var in df_original.columns:
        status = "❌ PRÉSENTE" if var in df_preprocessed.columns else "✅ SUPPRIMÉE"
        print(f"  {var}: {status}")
    else:
        print(f"  {var}: ⚠️  Absente du dataset original")

print("\n🏷️  Variables redondantes catégorielles (devaient être supprimées):")
for var in redundant_categorical:
    if var in df_original.columns:
        status = "❌ PRÉSENTE" if var in df_preprocessed.columns else "✅ SUPPRIMÉE"
        print(f"  {var}: {status}")
    else:
        print(f"  {var}: ⚠️  Absente du dataset original")

================================================================================
VÉRIFICATION DES SUPPRESSIONS
================================================================================

🔢 Variables ordinales redondantes (devaient être supprimées):
  ST005: ✅ SUPPRIMÉE
  ST007: ✅ SUPPRIMÉE
  ST253: ✅ SUPPRIMÉE
  ST255: ✅ SUPPRIMÉE
  ST097: ✅ SUPPRIMÉE

🏷️  Variables métadonnées catégorielles (devaient être supprimées):
  Option_CT: ✅ SUPPRIMÉE
  Option_FL: ✅ SUPPRIMÉE
  Option_ICTQ: ✅ SUPPRIMÉE
  Option_PQ: ✅ SUPPRIMÉE
  Option_TQ: ✅ SUPPRIMÉE
  Option_UH: ✅ SUPPRIMÉE
  Option_WBQ: ✅ SUPPRIMÉE
  CYC: ✅ SUPPRIMÉE
  NatCen: ✅ SUPPRIMÉE
  SUBNATIO: ✅ SUPPRIMÉE

🏷️  Variables redondantes catégorielles (devaient être supprimées):
  LANGTEST_PAQ: ✅ SUPPRIMÉE
  LANGTEST_QQQ: ✅ SUPPRIMÉE
  ST003D03T: ✅ SUPPRIMÉE
  ST001D01T: ✅ SUPPRIMÉE
  PA008: ✅ SUPPRIMÉE
  PA162: ✅ SUPPRIMÉE
  OCOD3: ✅ SUPPRIMÉE

### 3.2 Scores composites

In [7]:
print("\n" + "="*80)
print("VÉRIFICATION DES SCORES COMPOSITES")
print("="*80)

composite_scores = [
    ('Score_Support_Parental', ['PA003', 'ST300']),
    ('Score_Support_Enseignant', ['ST100', 'ST270'])
]

for score_name, source_vars in composite_scores:
    print(f"\n📊 {score_name}")
    
    # Vérifier présence des variables sources dans original
    sources_in_original = [v for v in source_vars if v in df_original.columns]
    print(f"  Variables sources dans original: {sources_in_original}")
    
    if score_name in df_preprocessed.columns:
        print(f"  ✅ Score créé")
        # Vérifier que les variables sources ont été supprimées
        sources_in_preprocessed = [v for v in source_vars if v in df_preprocessed.columns]
        if sources_in_preprocessed:
            print(f"  ⚠️  Variables sources encore présentes: {sources_in_preprocessed}")
        else:
            print(f"  ✅ Variables sources supprimées")
    else:
        if len(sources_in_original) >= 2:
            print(f"  ⚠️  Score NON créé (sources disponibles)")
        else:
            print(f"  ℹ️  Score NON créé (sources manquantes dans original)")


VÉRIFICATION DES SCORES COMPOSITES

📊 Score_Support_Parental
  Variables sources dans original: ['PA003', 'ST300']
  ⚠️  Score NON créé (sources disponibles)

📊 Score_Support_Enseignant
  Variables sources dans original: ['ST100', 'ST270']
  ⚠️  Score NON créé (sources disponibles)


### 3.3 Regroupement ISCO

In [8]:
print("\n" + "="*80)
print("VÉRIFICATION DES REGROUPEMENTS ISCO")
print("="*80)

isco_vars = ['OCOD1', 'OCOD2']

for var in isco_vars:
    grouped_var = f"{var}_grouped"
    
    print(f"\n📊 {var}")
    
    if var in df_original.columns:
        original_unique = df_original.select(pl.col(var).n_unique()).item()
        print(f"  Cardinalité originale: {original_unique}")
        
        if var in df_preprocessed.columns:
            print(f"  ❌ Variable originale encore présente")
        else:
            print(f"  ✅ Variable originale supprimée")
        
        if grouped_var in df_preprocessed.columns:
            grouped_unique = df_preprocessed.select(pl.col(grouped_var).n_unique()).item()
            print(f"  ✅ Variable groupée créée")
            print(f"  Cardinalité groupée: {grouped_unique}")
            print(f"  Réduction: {original_unique} → {grouped_unique} (-{(original_unique-grouped_unique)/original_unique*100:.1f}%)")
            
            # Afficher la distribution
            print(f"  Distribution:")
            distribution = df_preprocessed.group_by(grouped_var).agg(pl.count()).sort(grouped_var)
            print(distribution)
        else:
            print(f"  ❌ Variable groupée NON créée")
    else:
        print(f"  ⚠️  Absente du dataset original")


VÉRIFICATION DES REGROUPEMENTS ISCO

📊 OCOD1
  Cardinalité originale: 774
  ✅ Variable originale supprimée
  ✅ Variable groupée créée
  Cardinalité groupée: 11
  Réduction: 774 → 11 (-98.6%)
  Distribution:
shape: (11, 2)
┌───────────────┬────────┐
│ OCOD1_grouped ┆ count  │
│ ---           ┆ ---    │
│ i64           ┆ u32    │
╞═══════════════╪════════╡
│ null          ┆ 348119 │
│ 0             ┆ 292    │
│ 1             ┆ 43904  │
│ 2             ┆ 167601 │
│ 3             ┆ 64873  │
│ …             ┆ …      │
│ 5             ┆ 132603 │
│ 6             ┆ 12139  │
│ 7             ┆ 25148  │
│ 8             ┆ 9340   │
│ 9             ┆ 321833 │
└───────────────┴────────┘

📊 OCOD2
  Cardinalité originale: 775
  ✅ Variable originale supprimée
  ✅ Variable groupée créée
  Cardinalité groupée: 11
  Réduction: 775 → 11 (-98.6%)
  Distribution:


/var/folders/cn/f1c072zn49n5lbvp5frkrh500000gn/T/ipykernel_59414/1363085826.py:29: DeprecationWarning: `pl.count()` is deprecated. Please use `pl.len()` instead.
(Deprecated in version 0.20.5)
  distribution = df_preprocessed.group_by(grouped_var).agg(pl.count()).sort(grouped_var)


shape: (11, 2)
┌───────────────┬────────┐
│ OCOD2_grouped ┆ count  │
│ ---           ┆ ---    │
│ i64           ┆ u32    │
╞═══════════════╪════════╡
│ null          ┆ 348119 │
│ 0             ┆ 2680   │
│ 1             ┆ 80792  │
│ 2             ┆ 115788 │
│ 3             ┆ 73380  │
│ …             ┆ …      │
│ 5             ┆ 80139  │
│ 6             ┆ 32398  │
│ 7             ┆ 119909 │
│ 8             ┆ 67570  │
│ 9             ┆ 230950 │
└───────────────┴────────┘


## 4. Statistiques descriptives

In [9]:
print("\n" + "="*80)
print("STATISTIQUES DESCRIPTIVES DES COLONNES")
print("="*80)

# Colonnes numériques
numeric_cols = [col for col in df_preprocessed.columns if df_preprocessed[col].dtype in [pl.Int64, pl.Int32, pl.Float64, pl.Float32]]

if numeric_cols:
    print(f"\n📊 Variables numériques ({len(numeric_cols)} colonnes)")
    print(df_preprocessed.select(numeric_cols).describe())
else:
    print("\n⚠️  Aucune variable numérique détectée")


STATISTIQUES DESCRIPTIVES DES COLONNES

📊 Variables numériques (84 colonnes)
shape: (9, 85)
┌───────────┬───────────┬───────────┬───────────┬───┬───────────┬───────────┬───────────┬──────────┐
│ statistic ┆           ┆ Year      ┆ CNTRYID   ┆ … ┆ science_q ┆ science_q ┆ OCOD1_gro ┆ OCOD2_gr │
│ ---       ┆ ---       ┆ ---       ┆ ---       ┆   ┆ 6_total_t ┆ 7_total_t ┆ uped      ┆ ouped    │
│ str       ┆ f64       ┆ f64       ┆ f64       ┆   ┆ iming     ┆ iming     ┆ ---       ┆ ---      │
│           ┆           ┆           ┆           ┆   ┆ ---       ┆ ---       ┆ f64       ┆ f64      │
│           ┆           ┆           ┆           ┆   ┆ f64       ┆ f64       ┆           ┆          │
╞═══════════╪═══════════╪═══════════╪═══════════╪═══╪═══════════╪═══════════╪═══════════╪══════════╡
│ count     ┆ 1.172086e ┆ 1.172086e ┆ 1.172086e ┆ … ┆ 611049.0  ┆ 586121.0  ┆ 823967.0  ┆ 823967.0 │
│           ┆ 6         ┆ 6         ┆ 6         ┆   ┆           ┆           ┆           ┆          

================================================================================
STATISTIQUES DESCRIPTIVES DES COLONNES
================================================================================

📊 Variables numériques (84 colonnes)
shape: (9, 85)
┌───────────┬───────────┬───────────┬───────────┬───┬───────────┬───────────┬───────────┬──────────┐
│ statistic ┆           ┆ Year      ┆ CNTRYID   ┆ … ┆ science_q ┆ science_q ┆ OCOD1_gro ┆ OCOD2_gr │
│ ---       ┆ ---       ┆ ---       ┆ ---       ┆   ┆ 6_total_t ┆ 7_total_t ┆ uped      ┆ ouped    │
│ str       ┆ f64       ┆ f64       ┆ f64       ┆   ┆ iming     ┆ iming     ┆ ---       ┆ ---      │
│           ┆           ┆           ┆           ┆   ┆ ---       ┆ ---       ┆ f64       ┆ f64      │
│           ┆           ┆           ┆           ┆   ┆ f64       ┆ f64       ┆           ┆          │
╞═══════════╪═══════════╪═══════════╪═══════════╪═══╪═══════════╪═══════════╪═══════════╪══════════╡
│ count     ┆ 1.172086e ┆ 1.172086e ┆ 1.172086e ┆ … ┆ 611049.0  ┆ 586121.0  ┆ 823967.0  ┆ 823967.0 │
│           ┆ 6         ┆ 6         ┆ 6         ┆   ┆           ┆           ┆           ┆          │
│ null_coun ┆ 0.0       ┆ 0.0       ┆ 0.0       ┆ … ┆ 561037.0  ┆ 585965.0  ┆ 348119.0  ┆ 348119.0 │
│ t         ┆           ┆           ┆           ┆   ┆           ┆           ┆           ┆          │
│ mean      ┆ 878812.69 ┆ 2018.5172 ┆ 449.43761 ┆ … ┆ 68475.470 ┆ 65601.198 ┆ 5.633433  ┆ 5.664678 │
│           ┆ 5211      ┆ 24        ┆ 5         ┆   ┆ 86        ┆ 805       ┆           ┆          │
│ std       ┆ 507534.02 ┆ 2.835234  ┆ 272.91803 ┆ … ┆ 42776.513 ┆ 58513.368 ┆ 3.024921  ┆ 2.949263 │
│           ┆ 2377      ┆           ┆ 2         ┆   ┆ 326       ┆ 221       ┆           ┆          │
│ min       ┆ 0.0       ┆ 2015.0    ┆ 8.0       ┆ … ┆ 269.0     ┆ 447.66666 ┆ 0.0       ┆ 0.0      │
│           ┆           ┆           ┆           ┆   ┆           ┆ 7         ┆           ┆          │
│ 25%       ┆ 439061.0  ┆ 2015.0    ┆ 203.0     ┆ … ┆ 42943.332 ┆ 40690.5   ┆ 2.0       ┆ 3.0      │
│           ┆           ┆           ┆           ┆   ┆ 031       ┆           ┆           ┆          │
│ 50%       ┆ 878784.0  ┆ 2018.0    ┆ 411.0     ┆ … ┆ 63023.666 ┆ 59195.0   ┆ 5.0       ┆ 7.0      │
│           ┆           ┆           ┆           ┆   ┆ 667       ┆           ┆           ┆          │
│ 75%       ┆ 1.318404e ┆ 2022.0    ┆ 703.0     ┆ … ┆ 86796.5   ┆ 82685.664 ┆ 9.0       ┆ 9.0      │
│           ┆ 6         ┆           ┆           ┆   ┆           ┆ 062       ┆           ┆          │
│ max       ┆ 1.758128e ┆ 2022.0    ┆ 983.0     ┆ … ┆ 9.237028e ┆ 2.513825e ┆ 9.0       ┆ 9.0      │
│           ┆ 6         ┆           ┆           ┆   ┆ 6         ┆ 7         ┆           ┆          │
└───────────┴───────────┴───────────┴───────────┴───┴───────────┴───────────┴───────────┴──────────┘

In [10]:
# Distribution des valeurs manquantes par colonne
print("\n📉 TOP 10 COLONNES AVEC LE PLUS DE VALEURS MANQUANTES")

missing_by_col = []
for col in df_preprocessed.columns:
    null_count = df_preprocessed[col].null_count()
    null_pct = (null_count / len(df_preprocessed)) * 100
    missing_by_col.append({'column': col, 'null_count': null_count, 'null_pct': null_pct})

missing_df = pl.DataFrame(missing_by_col).sort('null_count', descending=True).head(10)
print(missing_df)


📉 TOP 10 COLONNES AVEC LE PLUS DE VALEURS MANQUANTES
shape: (10, 3)
┌──────────────────────────┬────────────┬───────────┐
│ column                   ┆ null_count ┆ null_pct  │
│ ---                      ┆ ---        ┆ ---       │
│ str                      ┆ i64        ┆ f64       │
╞══════════════════════════╪════════════╪═══════════╡
│ science_q7_total_timing  ┆ 585965     ┆ 49.993345 │
│ math_q9_total_timing     ┆ 585192     ┆ 49.927394 │
│ science_q8_average_score ┆ 579605     ┆ 49.450723 │
│ reading_q6_total_timing  ┆ 569049     ┆ 48.550106 │
│ math_q9_average_score    ┆ 566025     ┆ 48.292105 │
│ math_q8_total_timing     ┆ 563432     ┆ 48.070875 │
│ science_q6_total_timing  ┆ 561037     ┆ 47.866539 │
│ reading_q5_total_timing  ┆ 547013     ┆ 46.67004  │
│ science_q5_total_timing  ┆ 542649     ┆ 46.297712 │
│ math_q7_total_timing     ┆ 541417     ┆ 46.1926   │
└──────────────────────────┴────────────┴───────────┘


## 5. Résumé des validations

In [11]:
print("\n" + "="*80)
print("RÉSUMÉ DES VALIDATIONS")
print("="*80)

validations = []

# Check suppressions ordinales
ordinal_removed = sum(1 for v in redundant_ordinal if v in df_original.columns and v not in df_preprocessed.columns)
ordinal_expected = sum(1 for v in redundant_ordinal if v in df_original.columns)
validations.append(('Variables ordinales supprimées', ordinal_removed, ordinal_expected))

# Check suppressions métadonnées
metadata_removed = sum(1 for v in metadata_categorical if v in df_original.columns and v not in df_preprocessed.columns)
metadata_expected = sum(1 for v in metadata_categorical if v in df_original.columns)
validations.append(('Métadonnées supprimées', metadata_removed, metadata_expected))

# Check suppressions redondances catégorielles
redundant_removed = sum(1 for v in redundant_categorical if v in df_original.columns and v not in df_preprocessed.columns)
redundant_expected = sum(1 for v in redundant_categorical if v in df_original.columns)
validations.append(('Redondances catégorielles supprimées', redundant_removed, redundant_expected))

# Check regroupements ISCO
isco_grouped = sum(1 for v in isco_vars if f"{v}_grouped" in df_preprocessed.columns)
isco_expected = sum(1 for v in isco_vars if v in df_original.columns)
validations.append(('Regroupements ISCO créés', isco_grouped, isco_expected))

print("\n")
for name, actual, expected in validations:
    status = "✅" if actual == expected else "⚠️"
    print(f"{status} {name}: {actual}/{expected}")

print("\n" + "="*80)


RÉSUMÉ DES VALIDATIONS


✅ Variables ordinales supprimées: 5/5
✅ Métadonnées supprimées: 10/10
✅ Redondances catégorielles supprimées: 7/7
✅ Regroupements ISCO créés: 2/2

